<a href="https://colab.research.google.com/github/lalasray/custom_pose_capture/blob/main/Notebook/Facebook_VideoPose3D_Inference_in_the_wild_Detectron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Pytorch & Caffe2**

In [1]:
!wget https://anaconda.org/pytorch/pytorch/1.2.0/download/linux-64/pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!tar xvjf pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/

--2021-11-09 22:43:24--  https://anaconda.org/pytorch/pytorch/1.2.0/download/linux-64/pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a11922b393ec61208ae9668/5d4c44f5fe6b2a65c4c559c7?response-content-disposition=attachment%3B%20filename%3D%22pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20211109T224324Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEDYaCXVzLWVhc3QtMSJIMEYCIQD58bD2x%2Frl34FYTpp0RooI2qwZUfWJ4kTRxTEMjrXWeAIhAO%2B1Ie1LA3pmVKqQTklZbT5oPn7ltplrl0JjVCyVZiYIKoMECN7%2F%2F%2F%2F%2F%2F%2F

# check if Caffe2 was  build

In [2]:
# To check if Caffe2 build was successful
!python -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

# To check if Caffe2 GPU build was successful
!python -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'

Success
1


# Install COCO Dataset 

In [3]:
!apt-get install python-dev
!pip install cython
!pip install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make install

import os
os.environ['COCOAPI'] = ":/content/cocoapi"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 21.55 MiB/s, done.
Resolving deltas: 100% (576/576), done.
# install pycocotools to the Python site-packages
python setup.py build_ext install
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.

# Install Detectron

In [4]:
!git clone https://github.com/facebookresearch/detectron
!pip install -r detectron/requirements.txt
!cd detectron && make
!python detectron/detectron/tests/test_spatial_narrow_as_op.py

Cloning into 'detectron'...
remote: Enumerating objects: 1377, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1377 (delta 1), reused 2 (delta 0), pack-reused 1370
Receiving objects: 100% (1377/1377), 4.38 MiB | 17.96 MiB/s, done.
Resolving deltas: 100% (825/825), done.
     |████████████████████████████████| 375 kB 5.4 MB/s 
  Created wheel for pyyaml: filename=PyYAML-3.12-cp37-cp37m-linux_x86_64.whl size=43073 sha256=af29c0bb821acf11ac4d2cb295f33013579df38ef535aa5774a5be8f6483813f
  Stored in directory: /root/.cache/pip/wheels/21/cc/0a/8f691c938b079b07fa17a2e2940e3faae0a3dfb3ff5b3ac356
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
python setup.py develop --user
Compiling detectron/utils/cython_bbox.pyx because it changed.
Compiling detectron/utils/cython_nms.pyx because it changed.
[1/2] Cythoni

# Install VideoPose3D & Copy Video Script to Detectron Tools Folder

In [5]:
#copy file from VideoPose3d
!git clone https://github.com/facebookresearch/VideoPose3D
!cp VideoPose3D/inference/infer_video.py detectron/tools/infer_video.py

Cloning into 'VideoPose3D'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121
Receiving objects: 100% (121/121), 9.53 MiB | 24.64 MiB/s, done.
Resolving deltas: 100% (41/41), done.


# Download Pretrained Human3.6m Coco Model

In [6]:
!mkdir VideoPose3D/checkpoint
os.chdir('VideoPose3D/checkpoint')
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
os.chdir('../..')

--2021-11-09 22:45:49--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M  28.4MB/s    in 2.3s    

2021-11-09 22:45:51 (28.4 MB/s) - ‘pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



# Download Youtube Video for 3D Pose Estimation (specify YOUTUBE_ID)

In [7]:
YOUTUBE_ID ='UpH7rm0cYbM'


!pip install -q youtube-dl
#download video
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

!mkdir videos   
  
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:42 -t 00:00:05 videos/video.mp4

# recommendet alternation to 50fps 
#!ffmpeg -i videos/video.mp4 -filter "minterpolate='fps=50'" -crf 0 videos/video50fps.mp4

     |████████████████████████████████| 1.9 MB 5.4 MB/s 
[youtube] UpH7rm0cYbM: Downloading webpage
[download] Destination: youtube.mp4
[download] 100% of 11.24MiB in 02:54
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh -

# Compute 2D Coordinates with Detectron

In [8]:
!mkdir output
!python detectron/tools/infer_video.py \
    --cfg detectron/configs/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml \
    --output-dir output \
    --image-ext mp4 \
    --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \
   videos
    
#	  --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \

    

Found Detectron ops lib: /usr/local/lib/python3.7/dist-packages/torch/lib/libcaffe2_detectron_ops_gpu.so
[E init_intrinsics_check.cc:44] CPU feature avx is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:44] CPU feature avx2 is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
[E init_intrinsics_check.cc:44] CPU feature fma is present on your machine, but the Caffe2 binary is not compiled with it. It means you may not get the full speed of your CPU.
INFO io.py: 112: Downloading remote file https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl to /tmp/detectron-download-cache/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.ya

# Prepare Detectron Output to fit VideoPose3D Input

In [9]:

!mkdir ./VideoPose3D/data/detectronoutput
!cp output/video.mp4.npz VideoPose3D/data/detectronoutput/video.mp4.npz
os.chdir('VideoPose3D/data') # This script must be launched from the "data" directory
!python prepare_data_2d_custom.py -i detectronoutput -o myvideos
os.chdir('../../')

Parsing 2D detections from detectronoutput
Processing detectronoutput/video.mp4.npz
120 total frames processed
0 frames were interpolated
----------
Saving...
Done.


# Compute 3D Joints with VideoPose3D

In [10]:
#os.chdir('../')

#os.chdir('checkpoint')
#!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_cpn.bin
##!wget https://dl.fbaipublicfiles.com/video-pose-3d/d-pt-243.bin

!cp ./videos/video.mp4 VideoPose3D/video.mp4
os.chdir('VideoPose3D')


#!python run.py -d custom -k MyCustomDatasetName -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject S1 --viz-action custom --viz-camera 0 --viz-export My3dDataExport --viz-size 6
#!python run.py -d custom -k MyCustomDatasetName -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject  video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-size 6

#!python run.py -e 80 -k gt -arc 3,3,3,3,3


#!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --viz-export My3dDataExport

#!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate d-pt-243.bin --render --viz-subject video.mp4 --viz-action Directions --viz-video video.mp4 --viz-camera 0 --viz-output output_scater.mp4 --viz-size 5 --viz-downsample 1 --viz-skip 9

#!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-size 6
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-export outputfile --viz-size 6

#working version 
#!python run.py -k gt -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_cpn.bin --viz-export My3dDataExport --viz-output output.mp4

Namespace(actions='*', architecture='3,3,3,3,3', batch_size=1024, bone_length_term=True, by_subject=False, causal=False, channels=1024, checkpoint='checkpoint', checkpoint_frequency=10, data_augmentation=True, dataset='custom', dense=False, disable_optimizations=False, downsample=1, dropout=0.25, epochs=60, evaluate='pretrained_h36m_detectron_coco.bin', export_training_curves=False, keypoints='myvideos', learning_rate=0.001, linear_projection=False, lr_decay=0.95, no_eval=False, no_proj=False, render=True, resume='', stride=1, subjects_test='S9,S11', subjects_train='S1,S5,S6,S7,S8', subjects_unlabeled='', subset=1, test_time_augmentation=True, viz_action='custom', viz_bitrate=3000, viz_camera=0, viz_downsample=1, viz_export='outputfile', viz_limit=-1, viz_no_ground_truth=False, viz_output='output.mp4', viz_size=6, viz_skip=0, viz_subject='video.mp4', viz_video='video.mp4', warmup=1)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: 243 frames
INFO: Tr

# Display Results - Joint Export 

In [11]:
#inspect joints export 

import numpy as np
data  = np.load('outputfile.npy')
lst = data
for item in lst:
    print(item)
    


[[-1.8475839e-06 -1.1965542e-05  4.0793790e-07]
 [-1.1790417e-01  5.1899608e-03 -7.7377274e-03]
 [-8.3439991e-02  4.2249629e-01  4.4070065e-02]
 [-6.5079898e-02  8.3384895e-01  1.9296925e-01]
 [ 1.1790391e-01 -5.1964009e-03  7.7407956e-03]
 [ 1.0133499e-01  4.1356862e-01  4.6372682e-02]
 [ 7.7051595e-02  8.2238775e-01  2.0410264e-01]
 [-4.6358695e-03 -2.2140887e-01 -3.4013975e-02]
 [ 2.3637116e-02 -4.6765995e-01 -7.0715666e-02]
 [ 1.0986255e-02 -5.1801884e-01 -1.6546169e-01]
 [ 1.7456440e-02 -6.2121612e-01 -1.1946954e-01]
 [ 1.3754904e-01 -3.9311486e-01 -3.7176117e-02]
 [ 2.1584997e-01 -1.6375563e-01  5.2912425e-02]
 [ 2.3083806e-01 -3.2100573e-02 -1.2024416e-01]
 [-1.0151395e-01 -4.0736192e-01 -5.7327036e-02]
 [-2.1571527e-01 -1.9088960e-01  1.7824002e-02]
 [-1.9671004e-01 -4.4349529e-02 -1.6048478e-01]]
[[-2.12353007e-06 -1.16421579e-05  5.56902137e-07]
 [-1.17669381e-01  4.66090813e-03 -6.63584843e-03]
 [-8.35900754e-02  4.21773285e-01  4.44882736e-02]
 [-6.41645715e-02  8.32773507e

# Display Results - Joint Video

In [ ]:
#display video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

#Download Joint Export & Video

In [12]:
from google.colab import files


files.download('output.mp4')
files.download('outputfile.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>